In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j yfiles_jupyter_graphs pyPDF2 llama-index llama-index-llms-openai llama-index-graph-stores-neo4j llama-index-embeddings-openai llama-index-llms-azure-openai llama-index-vector-stores-chroma langchain_groq openai llama-index-embeddings-fastembed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from typing import List
from llama_index.core import (
    SimpleDirectoryReader, VectorStoreIndex, SummaryIndex, Settings,
    StorageContext, KnowledgeGraphIndex
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.llms.openai import OpenAI
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from IPython.display import Markdown, display

In [ ]:
NEO4J_URI= ""
NEO4J_USERNAME= ""
NEO4J_PASSWORD= ""
OPENAI_API_KEY= ""

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [ ]:
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 1024
Settings.llm = OpenAI(model="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [ ]:
def load_documents(file_path: str) -> List:
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    print(f"Loaded {len(documents)} documents")
    print(f"Document Metadata: {documents[0].metadata}")
    return documents

In [ ]:
def create_vector_index(nodes):
    return VectorStoreIndex(nodes)

In [ ]:
def create_summary_index(nodes):
    return SummaryIndex(nodes)

In [ ]:
def create_knowledge_graph_index(documents, graph_store):
    storage_context = StorageContext.from_defaults(graph_store=graph_store)

    graph_store.query("MATCH (n) DETACH DELETE n")

    return KnowledgeGraphIndex.from_documents(
        documents,
        storage_context=storage_context,
        max_triplets_per_chunk=2,
    )

In [ ]:
def create_router_query_engine(summary_index, vector_index):
    """Create a router query engine from summary and vector indexes."""
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    vector_query_engine = vector_index.as_query_engine()

    summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description="Useful for summarization questions related to the document"
    )
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description="Useful for retrieving specific context from the document"
    )

    return RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[summary_tool, vector_tool],
        verbose=True
    )

In [ ]:
def main():
    documents = load_documents('your_document')

    splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=100)
    nodes = splitter.get_nodes_from_documents(documents)
    print(f"Created {len(nodes)} nodes")

    vector_index = create_vector_index(nodes)
    summary_index = create_summary_index(nodes)

    graph_store = Neo4jGraphStore(
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        url=NEO4J_URI,
    )

    kg_index = create_knowledge_graph_index(documents, graph_store)

    router_query_engine = create_router_query_engine(summary_index, vector_index)

    rag_response = router_query_engine.query("Question-1")
    print("RAG Response:")
    display(Markdown(f"<b>{rag_response}</b>"))
    print(f"Number of source nodes: {len(rag_response.source_nodes)}")

    kg_query_engine = kg_index.as_query_engine(include_text=False, response_mode="tree_summarize")
    kg_response = kg_query_engine.query("Question-2")
    print("\nKnowledge Graph Response:")
    display(Markdown(f"<b>{kg_response}</b>"))

if __name__ == "__main__":
    main()

ERROR:neo4j.io:Failed to write data to connection IPv4Address(('48e16e9c.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.114.186', 7687)))
ERROR:neo4j.io:Failed to write data to connection ResolvedIPv4Address(('34.126.114.186', 7687)) (ResolvedIPv4Address(('34.126.114.186', 7687)))


Loaded 82 documents
Document Metadata: {'page_label': '1', 'file_name': '2022-sustainability-report.pdf', 'file_path': 'data/2022-sustainability-report.pdf', 'file_type': 'application/pdf', 'file_size': 8330454, 'creation_date': '2024-08-08', 'last_modified_date': '2024-08-08'}
Created 104 nodes



Knowledge Graph Response:


<b>Amazon has taken significant steps towards promoting gender equity in their supply chains. They have focused on increasing the number of women in executive positions, particularly in STEM-focused roles. By prioritizing safety and advocating for gender diversity, Amazon aims to create a more inclusive and equal work environment for women across their supply chain operations.

In addition to internal initiatives, Amazon has partnered with external organizations and certifications to further support gender equity efforts. By working with diverse suppliers and leveraging external certifications, Amazon is actively engaging in activities that promote and ensure gender equality within their supply chains.

Furthermore, Amazon has committed to providing access to critical health care services and opportunities for women in their supply chains. By offering support for responsible supply chains and aiming to increase the number of women in executive positions, Amazon is demonstrating a strong commitment to advancing gender equity and empowering women within their supply chain network.</b>